In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
import pickle as pkl
from learn2learn.vision.datasets import MiniImagenet
from tqdm.notebook import tqdm
from attack import attack, parse_param, test_model
from utils import get_device, caculate_param_remove
import random
device = get_device()


In [2]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


setup_seed(3407)


In [3]:
from datasets import load_cifar10
from models.resnet import load_cifar10_resnet50
model = load_cifar10_resnet50()


In [4]:
all_param_names = list()
for name, param in model.named_parameters():
    if not "bn" in name and not "shortcut.1" in name:
        all_param_names.append(name)
all_param_names = all_param_names[:-2]


In [5]:
train_loaders, test_dataloaders, train_dataloader_all, test_dataloader_all = load_cifar10()


Files already downloaded and verified
Files already downloaded and verified


In [6]:
def train_eval(net, train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all, epochs=50, lr=0.001):
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)
    for epoch in range(epochs):
        net.train()
        for x, y in tqdm(train_dataloader_all, total=len(train_dataloader_all)):
            x, y = x.to(device), y.to(device)
            outputs = net(x)
            loss = loss_func(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        net.eval()
        accs = list()
        preds,labels = test_model(net, test_dataloader_all)
        acc = np.mean((preds == labels).astype(np.float32))
        print("epoch: {}, acc: {}".format(epoch, acc))
        all_totals = list()
        for i in range(10):
            all_totals.append(attack(train_loaders[i], all_param_names,
                            net, norm=True, alpha=0.00001, num_steps=4, op="add"))

        thre = 0.9
        param_remove = caculate_param_remove(all_param_names, all_totals, net, thre)
        for param in all_param_names:
            param_ = parse_param(param)
            try:
                exec("net." + param_ + "[~param_remove[param]] = 0")
            except:
                exec("net." + param_ + "[~param_remove[param],:] = 0")

In [7]:
train_eval(model, train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all, epochs=50, lr=0.001)

  0%|          | 0/196 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)